# Installing Libraries

In [ ]:
# Google Play Store Scraper Library
!pip install google-play-scraper
# Repository: https://github.com/JoMingyu/google-play-scraper

In [ ]:
# Apple App Store Review Scraper
!pip install app_store_scraper
# Repository: https://github.com/cowboy-bebug/app-store-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into a

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from app_store_scraper import AppStore
from google_play_scraper import app, Sort, reviews_all

# Google Play Store Scraper

In [ ]:
# GXS Mobile App link https://play.google.com/store/apps/details?id=sg.com.gxs.app&hl=en&gl=US
App_ID = 'sg.com.gxs.app'

In [ ]:
# Getting App Details & making sure App_ID is correct
result = app(
    App_ID,
    lang='en', # defaults to 'en'
    country='sg' # defaults to 'us'
)

In [ ]:
# Checking if App_ID corresponds to the correct app
print(result)

{'title': 'GXS Bank', 'description': "Bank better with GXS, Singapore's coolest digibank. \r\n\r\nNeed a flexi line of credit that’s as flexible as your yoga moves? Say no more fam. Apply for a GXS FlexiLoan with just a few taps and get cash in minutes†. It’s like magic!\r\n\r\nReady to slay your saving goals? Easy! With the GXS Savings Account today, create multiple Saving Pockets and get attractive returns on your savings with interest credited daily! Plus, get your hands on the GXS Debit Card today! Pick from 3 cool card designs and score instant unlimited rewards* on eligible transactions of at least S$10! Don't wait, start swiping now!\r\n____________________________________________________\r\n\r\nTHE SIMPLEST WAY TO SAVE WITH GXS SAVINGS ACCOUNT\r\n- Create multiple Saving Pockets for everything you love and earn an attractive interest rate, credited daily. It makes saving for the things you love so much easier! \r\n- No minimum balance, no lock-ins, just pure savings fun.\r\n- A

565 Reviews on Google Play Store as of 4/3/2024

Reviews are split into 2 categories\
Ratings - only star ratings and no comments\
Reviews - star rating and comments

Ratings are unable to be scraped

In [ ]:
# Scraping Singapore Reviews
from google_play_scraper import Sort, reviews_all

sg_reviews = reviews_all(
    App_ID,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='sg', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

In [ ]:
# Check total number of reviews scraped
print(len(sg_reviews))

199


In [ ]:
# Creating Dataframe from Reviews
df_reviews = pd.DataFrame(np.array(sg_reviews),columns=['review'])
df_reviews = df_reviews.join(pd.DataFrame(df_reviews.pop('review').tolist()))

In [ ]:
df_reviews.head()
# Columns to keep (standardised format), rename and re-order columns
# Add new column Bank to label which bank the reviews are for
# No., UserName, Review, Score, Date, Bank

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,6caa76db-0b0e-4e9a-ab66-45a5c6935d2a,E,https://play-lh.googleusercontent.com/a-/ALV-U...,A feature they should implement is to save you...,3,12,2.23.0,2024-03-04 04:23:26,"Hi EE, that's a fantastic suggestion and it's ...",2024-03-04 05:00:32,2.23.0
1,36d50466-6d61-49a5-b64c-a30a15871877,Baggio Pan,https://play-lh.googleusercontent.com/a/ACg8oc...,Ok Good system.,5,0,2.23.0,2024-03-01 09:52:59,Hi Baggio! Thank you for your 5-star review. 💜...,2024-03-01 12:01:22,2.23.0
2,fdd0b0bd-3247-4580-ac71-febcee9a7c3c,VELAN Ram,https://play-lh.googleusercontent.com/a-/ALV-U...,good 👍,3,0,2.23.0,2024-02-24 05:22:26,Hi Velan! Thank you for your positive feedback...,2024-02-24 06:05:37,2.23.0
3,f16e96ed-a3dc-401b-8618-c4c7cf1d576c,Yusri Wagiman,https://play-lh.googleusercontent.com/a-/ALV-U...,Can not open,1,0,2.23.0,2024-02-24 03:52:03,"Hi Yusri, we're sorry to hear this. To better ...",2024-02-24 04:28:56,2.23.0
4,7aaa07f2-f578-4c43-80c9-62595d8eef73,Aaro,https://play-lh.googleusercontent.com/a/ACg8oc...,Hi any chance for foreign to apply? Im work at sg,1,0,2.22.0,2024-02-18 12:20:44,"Hi Aaro, thank you for your interest in our se...",2024-02-18 12:35:26,2.22.0


# App Store Review Scraper

In [ ]:
# Apple App Store Scraper
# 520 Ratings on Apple App Store as of 4/3/2024
# https://apps.apple.com/sg/app/gxs-bank/id1632183616
GXS_Bank = AppStore(country='sg', app_name='GXS Bank', app_id = '1632183616')

520 Ratings on Apple Play Store as of 4/3/2024

Reviews are split into 2 categories\
Ratings - only star ratings and no comments\
Reviews - star rating and comments

Ratings are not able to be scraped by this library

In [ ]:
GXS_Bank.review()
#if how_many is not provided, review() will terminate after all reviews are fetched.

In [ ]:
# Check how many reviews scrapped
GXS_Bank.reviews_count

128

In [ ]:
# Creating Dataframe from Reviews
data1 = pd.DataFrame(np.array(GXS_Bank.reviews),columns=['review'])
data2 = data1.join(pd.DataFrame(data1.pop('review').tolist()))

In [ ]:
data2 = data2.sort_values('date', ascending=False)
data2.head()
# Columns to keep (standardised format) rename and re-order columns if needed
# Add new column Bank to label which bank the reviews are for
# No., UserName, Review, Score, Date, Bank

,date,developerResponse,review,rating,isEdited,title,userName
41,2024-02-25 07:24:03,"{'id': 42216759, 'body': 'Hi there! Thank you ...",The app is easy to navigate and optimise for p...,5,False,Easy to use with friendly UIUX,Tiny Particles
19,2024-02-24 06:12:02,"{'id': 42187814, 'body': 'Hey there! We're sor...",Extremely disappointed with the GXS app. Despi...,1,False,Bad experience of this GXS app. Think again if...,HetTienDownGameBanQuyen
66,2024-02-21 14:03:59,"{'id': 42155008, 'body': 'Hi there. We're sorr...",Show allow to increase amount not reduce and s...,1,False,Cheater!,Nrhdxtan
31,2024-02-17 11:21:01,"{'id': 42034671, 'body': 'We're thrilled to he...",User friendly,5,False,Reminder n easy to use,YSL Alan
30,2024-02-10 03:22:39,"{'id': 41887361, 'body': 'Aw, thanks for the k...",This app is easy to use.,5,False,Smart & easy to use online bank,OnTheRoad9988


# Comparing with other digital banks
Scrap Google Play Store & App Store for the following banks
1. GXS Bank
2. Revolut
3. Wise
4. MariBank
5. ANEXT Bank
6. Trust Bank

# Combine all scraped datasets with standardised format
No., UserName, Review, Score, Date, Bank